In [ ]:
%matplotlib inline


# Global Akida workflow

.. Note:: Please refer to the TensorFlow  [tf.keras.models](https://www.tensorflow.org/api_docs/python/tf/keras/models)_
          module for model creation/import details and the [TensorFlow Guide](https://www.tensorflow.org/guide)_ for TensorFlow usage.

## 1. Create and train




### Imports

In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from keras.optimizers import Adam

2024-11-12 16:17:00.144045: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 16:17:00.532630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 16:17:00.532799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 16:17:00.600123: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 16:17:00.743511: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-12 16:17:00.745345: I tensorflow/core/platform/cpu_feature_guard.cc:1

### 1.1. Load and reshape projection dataset



In [ ]:
# import numpy as np

import matplotlib.cm as cm
import matplotlib.pyplot as plt

#TODO: import the dataset

#TODO: Load projection dataset
# x_test, y_test = ...

# Display a few images from the test set
f, axarr = plt.subplots(1, 4)
for i in range(0, 4):
    pass
    #TODO: adjust the following line to display the images
    # axarr[i].imshow(x_test[i].reshape((28, 28)), cmap=cm.Greys_r)
    # axarr[i].set_title('Class %d' % y_test[i])
plt.show()

### 1.2. Model definition

Note that at this stage, there is nothing specific to the Akida IP.
The model constructed below, as inspired by [this example](https://www.tensorflow.org/model_optimization/guide/quantization/training_example#train_a_model_for_mnist_without_quantization_aware_training)_,
is a completely standard [Keras](https://www.tensorflow.org/api_docs/python/tf/keras)_ CNN model.




In [2]:
### DEFINE MODEL
base_model = keras.applications.MobileNet(include_top=False, input_shape=(240, 240, 2), weights=None)
# base_model.trainable = False  # Freeze all layers in the base model

x = base_model.output
x = tf.keras.layers.Flatten()(x)
output_l = tf.keras.layers.Dense(7, activation='linear')(x)
model_keras = Model(inputs=base_model.input, outputs=output_l)

print(model_keras.summary())

/home/jskupien/.local/lib/python3.10/site-packages/keras/src/applications/mobilenet.py:207: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 2)]     0         
                                                                 
 conv1 (Conv2D)              (None, 120, 120, 32)      576       
                                                                 
 conv1_bn (BatchNormalizati  (None, 120, 120, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 120, 120, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 120, 120, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 120, 120, 32)      128   

### 1.3. Model training

Given the model created above, train the model and check its accuracy. The model should achieve
a test accuracy over 98% after 10 epochs.




In [ ]:
model_keras.compile(
    loss=None, #TODO: define the loss function
    optimizer=Adam(learning_rate=1e-3),
    metrics=['accuracy'])

_ = model_keras.fit(x_train, y_train, epochs=10, validation_split=0.1)

In [ ]:
score = model_keras.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])

## 2. Quantize



### 2.1. 8-bit quantization

An Akida accelerator processes 8 or 4-bits integer activations and weights. Therefore,
the floating point Keras model must be quantized in preparation to run on an Akida accelerator.

The QuantizeML [quantize](../../api_reference/quantizeml_apis.html#quantizeml.models.quantize)_
function can be used to quantize a Keras model for Akida. For this step in this example, an
“8/8/8” quantization scheme will be applied to the floating point Keras model to produce 8-bit
weights in the first layer, 8-bit weights in all other layers, and 8-bit activations.

The quantization process results in a Keras model with custom [QuantizeML quantized layers](../../api_reference/quantizeml_apis.html#layers)_ substituted for the original Keras layers.
All Keras API functions can be applied on this new model: ``summary()``, ``compile()``, ``fit()``. etc.

.. Note:: The ``quantize`` function applies [several transformations](../../api_reference/quantizeml_apis.html#transforms)_ to
          the original model. For example, it folds the batch normalization layers into the
          corresponding neural layers. The new weights are computed according to this folding
          operation.



In [6]:
from quantizeml.models import quantize, QuantizationParams

qparams = QuantizationParams(input_weight_bits=8, weight_bits=8, activation_bits=8)
model_quantized = quantize(model_keras, qparams=qparams)
model_quantized.summary()

2024-11-12 16:31:27.591525: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117964800 exceeds 10% of free system memory.


1024/1024 [==============================] - 26s 24ms/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 2)]     0         
                                                                 
 conv1 (QuantizedConv2D)     (None, 120, 120, 32)      608       
                                                                 
 conv1_relu (QuantizedReLU)  (None, 120, 120, 32)      64        
                                                                 
 conv_dw_1 (QuantizedDepthw  (None, 120, 120, 32)      320       
 iseConv2D)                                                      
                                                                 
 conv_dw_1_relu (QuantizedR  (None, 120, 120, 32)      64        
 eLU)                                                            
                                                                 
 c

.. Note:: Note that the number of parameters for the floating and quantized models differs,
          a consequence of the BatchNormalization folding and the additional parameters
          added for quantization. For further details, please refer to their respective summary.




Check the quantized model accuracy.



In [ ]:
def compile_evaluate(model):
    """ Compiles and evaluates the model, then return accuracy score. """
    model.compile(metrics=['accuracy'])
    return model.evaluate(x_test, y_test, verbose=0)[1]


print('Test accuracy after 8-bit quantization:', compile_evaluate(model_quantized))

### 2.2. Effect of calibration

The previous call to ``quantize`` was made with random samples for calibration
(default parameters). While the observed drop in accuracy is minimal, that is
around 1%, it can be worse on more complex models. Therefore, it is advised to
use a set of real samples from the training set for calibration during a call
to ``quantize``.
Note that this remains a calibration step rather than a training step in that
no output labels are required. Furthermore, any relevant data could be used for
calibration. The recommended settings for calibration that are widely used to
obtain the [zoo performance](../../model_zoo_performance.html#akida-2-0-models)_ are:

- 1024 samples
- a batch size of 100
- 2 epochs



In [ ]:
model_quantized = quantize(model_keras, qparams=qparams,
                           samples=x_train, num_samples=1024, batch_size=100, epochs=2)

Check the accuracy for the quantized and calibrated model.



In [ ]:
print('Test accuracy after calibration:', compile_evaluate(model_quantized))

Calibrating with real samples on this model recovers the initial float accuracy.



### 2.3. 4-bit quantization

The accuracy of the 8/8/8 quantized model is equal to that of the Keras floating point
model. In some cases, a smaller memory size for the model is required. This can be
accomplished through quantization of the model to smaller bitwidths.

The model will now be quantized to 8/4/4, that is 8-bit weights in the first layer with
4-bit weights and activations in all other layers. Such a quantization scheme will usually
introduce a performance drop.




In [ ]:
qparams = QuantizationParams(input_weight_bits=8, weight_bits=4, activation_bits=4)
model_quantized = quantize(model_keras, qparams=qparams,
                           samples=x_train, num_samples=1024, batch_size=100, epochs=2)

Check the 4-bit quantized accuracy.



In [ ]:
print('Test accuracy after 4-bit quantization:', compile_evaluate(model_quantized))

### 2.4. Model fine tuning (Quantization Aware Training)

When a model suffers from an accuracy drop after quantization, fine tuning or Quantization
Aware Training (QAT) may recover some or all of the original performance.

Note that since this is a fine tuning step, both the number of epochs and learning rate are
expected to be lower than during the initial float training.




In [ ]:
model_quantized.compile(
    loss=None, #TODO: define the loss function
    optimizer=Adam(learning_rate=1e-4),
    metrics=['accuracy'])

model_quantized.fit(x_train, y_train, epochs=5, validation_split=0.1)

In [ ]:
score = model_quantized.evaluate(x_test, y_test, verbose=0)[1]
print('Test accuracy after fine tuning:', score)

## 3. Convert




### 3.1 Convert to Akida model

When the quantized model produces satisfactory performance, it can be converted to the native
Akida format. The [convert](../../api_reference/cnn2snn_apis.html#cnn2snn.convert)_ function
returns a model in Akida format ready for inference.

As with Keras, the summary() method provides a textual representation of the Akida model.




In [7]:
from cnn2snn import convert

model_akida = convert(model_quantized)
model_akida.summary()

/home/jskupien/.local/lib/python3.10/site-packages/cnn2snn/quantizeml/blocks.py:160: UserWarning: Conversion stops at layer conv_pw_11_relu because of a dequantizer. The end of the model is ignored:
___________________________________________________
Layer (type)
conv_pad_12 (ZeroPadding2D)
conv_dw_12 (DepthwiseConv2D)
conv_dw_12_relu (ReLU)
conv_pw_12 (Conv2D)
conv_pw_12_relu (ReLU)
conv_dw_13 (DepthwiseConv2D)
conv_dw_13_relu (ReLU)
conv_pw_13 (Conv2D)
conv_pw_13_relu (ReLU)
flatten (Flatten)
dense (Dense)

  warnings.warn("Conversion stops" + stop_layer_msg + " because of a dequantizer. "


                  Model Summary                  
_________________________________________________
Input shape    Output shape   Sequences  Layers
[240, 240, 2]  [15, 15, 512]  1          25    
_________________________________________________

_____________________________________________________________________________
Layer (type)                               Output shape    Kernel shape    

============== SW/conv1-conv_pw_11_relu/dequantizer (Software) ==============

conv1 (Conv2D)                             [120, 120, 32]  (3, 3, 2, 32)   
_____________________________________________________________________________
conv_dw_1 (DepthwiseConv2D)                [120, 120, 32]  (3, 3, 32, 1)   
_____________________________________________________________________________
conv_pw_1 (Conv2D)                         [120, 120, 64]  (1, 1, 32, 64)  
_____________________________________________________________________________
conv_dw_2 (DepthwiseConv2D)                [60, 60, 64]  

### 3.2. Check performance



In [ ]:
accuracy = model_akida.evaluate(x_test, y_test)
print('Test accuracy after conversion:', accuracy)

# For non-regression purposes
assert accuracy > 0.96

### 3.3 Show predictions for a single image

Display one of the test images, such as the first image in the dataset from above, to visualize
the output of the model.




In [ ]:
# Test a single example
sample_image = 0
image = x_test[sample_image]
outputs = model_akida.predict(image.reshape(1, 28, 28, 1))
print('Input Label: %i' % y_test[sample_image])

f, axarr = plt.subplots(1, 2)
#TODO: adjust the following line to display the image
# axarr[0].imshow(x_test[sample_image].reshape((28, 28)), cmap=cm.Greys_r)
axarr[0].set_title('Class %d' % y_test[sample_image])
axarr[1].bar(range(10), outputs.squeeze())
axarr[1].set_xticks(range(10))
plt.show()

print(outputs.squeeze())

Consider the output from the model above. As is typical in backprop-trained models, the final
layer is a Dense layer with one neuron for each of the 10 classes in the dataset. The goal of
training is to maximize the response of the neuron corresponding to the label of each training
sample while minimizing the responses of the other neurons.

In the bar chart above, you can see the outputs from all 10 neurons. It is easy to see that neuron
7 responds much more strongly than the others. The first sample is indeed a number 7.


